# 01_Function calling 활용 방법

## 1. OpenAI 라이브러리 설치 확인

In [ ]:
# !pip install openai python-dotenv

## 2. OpenAI 정보 읽기

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
dotenv_path = find_dotenv(filename='./.env')
load_dotenv(dotenv_path=dotenv_path)

openai.api_type = "azure"
openai.api_version = os.getenv("OPENAI_API_VERSION","").strip()

API_KEY = os.getenv("OPENAI_API_KEY","").strip()
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"
openai.api_base = RESOURCE_ENDPOINT

deployment_id=os.getenv('DEPLOYMENT_NAME')

## 3. Function Calling 기본 이해

### Function Calling 설정

- 사용자 쿼리와 functions 매개변수에 정의된 함수 집합을 사용하여 모델을 호출
- 모델은 Function 호출 여부를 선택
- 함수가 호출되면 콘텐츠는 문자열화된 JSON 객체 포함
- 만들어야 하는 Function Call 과 인수(arguments)는 `response['choices'][0]['message']['function_call']`에 위치

In [ ]:
def get_function_call(messages, function_call = "auto"):
    functions = [
        {
            "name": "get_current_weather",
            "description": "해당 지역의 현재 날씨를 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "지역(도시) 이름, e.g. 서울시 동작구",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["섭씨", "화씨"]},
                },
                "required": ["location", "unit"],
            },
        },
    ]

    response = openai.ChatCompletion.create(
        deployment_id = deployment_id,
        messages=messages,
        functions=functions,
        function_call=function_call, 
    )
    return response

### 3가지 Function Calling 사용 방법

In [ ]:
first_message = [{"role": "user", "content": "서울 동작구의 날씨는 어때?"}]
# 'auto' : Let the model decide what function to call
print("모델이 호출하려는 function을 자동으로 결정:")
response = get_function_call(first_message, "auto")
print (response)
# print (json.dumps(response['choices'][0]['message']['function_call'], ensure_ascii=False, indent=4))

# 'none' : Don't call any function 
print("모델이 어떤 function도 호출하지 않음:")
response = get_function_call(first_message, "none")
print (response)

# force a specific function call
print("모델이 강제로 지정한 function을 호출:")
response = get_function_call(first_message, function_call={"name": "get_current_weather"})
print (response)
# print (json.dumps(response['choices'][0]['message']['function_call'], ensure_ascii=False, indent=4))


## 4. Function Calling 실제 적용

### `Function #1`: Get current time

In [ ]:
import pytz
from datetime import datetime

def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%Y-%m-%d, %H:%M:%S")

        return current_time
    except:
        return "죄송합니다. 해당 지역의 TimeZone을 찾을 수 없습니다."

In [ ]:
get_current_time("Asia/Seoul")

### `Function #2`: Get stock market data

In [ ]:
import pandas as pd
import json

def get_stock_market_data(index):
    available_indices = ["S&P 500", "NASDAQ Composite", "Dow Jones Industrial Average", "Financial Times Stock Exchange 100 Index"]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv('../data/stock_data.csv')
    
    # Filter data for the given index
    data_filtered = data[data['Index'] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=['Index'])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict, indent=4)


In [ ]:
print(get_stock_market_data("NASDAQ Composite"))

### `Function #3`: Calculator

In [ ]:
import math

def calculator(num1, num2, operator):
    if operator == '+':
        return str(num1 + num2)
    elif operator == '-':
        return str(num1 - num2)
    elif operator == '*':
        return str(num1 * num2)
    elif operator == '/':
        return str(num1 / num2)
    elif operator == '**':
        return str(num1 ** num2)
    elif operator == 'sqrt':
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [ ]:
print(calculator(5, 5, '+'))

### 4.1 모델이 호출 방법을 알 수 있도록 함수 설명

In [ ]:
functions = [
        {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            # "description": "이 지역의 현재 시간을 알려줘.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like Asia/Seoul, America/New_York, Asia/Bangkok, Europe/London",
                        # "description": "location 이름. pytz는 해당 위치의 시간대를 가져오는 데 사용됩니다. location 이름은 Asia/Seoul, America/New_York, Asia/Bangkok, Europe/London과 같은 형식이어야 합니다.",
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "get_stock_market_data",
            "description": "Get the stock market data for a given index",
            # "description": "주어진 인덱스에 대한 주식 시장 데이터 가져오기",
            "parameters": {
                "type": "object",
                "properties": {
                    "index": {
                        "type": "string",
                        "enum": ["S&P 500", "NASDAQ Composite", "Dow Jones Industrial Average", "Financial Times Stock Exchange 100 Index"]},
                },
                "required": ["index"],
            },    
        },
        {
            "name": "calculator",
            "description": "A simple calculator used to perform basic arithmetic operations",
            # "description": "기본 산술 연산을 수행하는 데 사용되는 간단한 계산기",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {"type": "string", "enum": ["+", "-", "*", "/", "**", "sqrt"]},
                },
                "required": ["num1", "num2", "operator"],
            },
        }
    ]

available_functions = {
            "get_current_time": get_current_time,
            "get_stock_market_data": get_stock_market_data,
            "calculator": calculator,
        } 

### 4.2 function call을 검증하는 helper 함수 정의

In [ ]:
import inspect

# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided 
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [ ]:
def run_conversation(messages, functions, available_functions, deployment_id):

    # Step 1: send the conversation and available functions to GPT
    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
    )
    response_message = response["choices"][0]["message"]


    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        print("Recommended Function call:")
        print(response_message.get("function_call"))
        print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        print("Output of function call:")
        print(function_response)
        print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        # for message in messages:
        #     print(message)
        # print()
        print(json.dumps(messages, ensure_ascii=False, indent=4))
        print()

        second_response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id
        )  # get a new response from GPT where it can see the function response

        return second_response

In [ ]:
messages = [{"role": "user", "content": "시애틀의 현재 시간은?"}]
assistant_response = run_conversation(messages, functions, available_functions, deployment_id)
print(assistant_response['choices'][0]['message'])
json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False, indent=4)

## 5. 동시 다수의 Function Calling 활용

In [ ]:
def run_multiturn_conversation(messages, functions, available_functions, deployment_id):
    # Step 1: send the conversation and available functions to GPT

    response = openai.ChatCompletion.create(
        deployment_id=deployment_id,
        messages=messages,
        functions=functions,
        function_call="auto", 
        temperature=0
    )

    # Step 2: check if GPT wanted to call a function
    while response["choices"][0]["finish_reason"] == 'function_call':
        response_message = response["choices"][0]["message"]
        # print("Recommended Function call:")
        # print(response_message.get("function_call"))
        # print()
        
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        
        function_name = response_message["function_call"]["name"]
        
        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        fuction_to_call = available_functions[function_name]  
        
        # verify function has correct number of arguments
        function_args = json.loads(response_message["function_call"]["arguments"])
        if check_args(fuction_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = fuction_to_call(**function_args)
        
        # print("Output of function call:")
        # print(function_response)
        # print()
        
        # Step 4: send the info on the function call and function response to GPT
        
        # adding assistant response to messages
        messages.append(
            {
                "role": response_message["role"],
                "name": response_message["function_call"]["name"],
                "content": response_message["function_call"]["arguments"],
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in next request:")
        # for message in messages:
        #     print(message)
        print(json.dumps(messages, ensure_ascii=False, indent=4))
        print()

        response = openai.ChatCompletion.create(
            messages=messages,
            deployment_id=deployment_id,
            function_call="auto",
            functions=functions,
            temperature=0
        )  # get a new response from GPT where it can see the function response

    return response

In [ ]:
# Can add system prompting to guide the model to call functions and perform in specific ways
# next_messages = [{"role": "system", "content": "Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users."}]
# next_messages.append({"role": "user", "content": "How much did S&P 500 change between July 12 and July 13? Use the calculator."})
next_messages = [{"role": "system", "content": "너는 사용자의 질문에 대한 답을 얻을 수 있도록 도와주는 유용한 어시스턴트입니다. 어시스턴트는 여러 도구에 액세스할 수 있으며 경우에 따라 사용자의 답변을 얻기 위해 여러 도구를 순서대로 호출해야 할 수도 있습니다."}]
next_messages.append({"role": "user", "content": "S&P 500은 7월 12일에서 7월 13일에 얼만큼 변경되었습니까? 계산기를 사용하십시오."})

assistant_response = run_multiturn_conversation(next_messages, functions, available_functions, deployment_id)
print("Final Response:")
print(assistant_response["choices"][0]["message"])
print(json.dumps(assistant_response['choices'][0]['message']["content"], ensure_ascii=False))
print("Conversation complete!")  